In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt
import seaborn as sns

cm = sns.light_palette("blue", as_cmap=True)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load the data

In [ ]:
df_match=pd.read_csv('/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv')
print(df_match.shape)
df_bbb=pd.read_csv('/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')
print(df_bbb.shape)

In [ ]:
df_match['yr']=df_match.date.str[:4]
df_match.head()

# Winners

In [ ]:
df_match.pivot_table(index='yr',columns=['winner'],aggfunc='count',values='venue').fillna('')

In [ ]:
alt.Chart(df_match).mark_circle().encode(
    x='winner',
    y='yr',
    size='count()',
    color='yr',
    tooltip=['count()']
    )

# ball level

In [ ]:
df_bbb.head()

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]
df_bbb.pivot_table(index='over',aggfunc='count',columns='ball',values='player_dismissed').style.apply(highlight_max).format("{:.1f}")

In [ ]:
df_bbb.pivot_table(index='batsman',aggfunc='count',columns='ball',values='player_dismissed'
                ).sort_values(by=1,ascending=False).head(30
                ).style.apply(highlight_max).format("{:.1f}")

In [ ]:
df_=df_bbb.pivot_table(index='batsman',aggfunc='count',columns='batsman_runs',values='ball'
                  ).sort_values(by=4,ascending=False)
df_.head(30).style.format("{:.1f}").background_gradient(cmap=cm).apply(highlight_max)
df_.columns=df_.columns.map(lambda x:'r'+str(x))
# alt.Chart(df_).mark_area().encode(
# x='batsman',
# y='batsman_runs'
# )
df_.fillna(0)

In [ ]:
#run outcome for balls
alt.Chart(df_.head(30).reset_index()).mark_bar().transform_fold(
    df_.columns.values,
).encode(
    x=alt.X("batsman:N",sort="-y"),
    y=alt.Y("value:Q"),
    color="key:N",
    tooltip=["batsman:N",'key:N','value:Q']
)

In [ ]:
#run for balls
df_.r0=0
df_.r2=df_.r2*2
df_.r3=df_.r3*3
df_.r4=df_.r4*4
df_.r5=df_.r5*6
df_.r6=df_.r6*6
alt.Chart(df_.head(30).reset_index()).mark_bar().transform_fold(
    df_.columns.values,
).encode(
    x=alt.X("batsman:N",sort="-y"),
    y=alt.Y("value:Q"),
    color="key:N",
    tooltip=["batsman:N",'key:N','value:Q']
)

# Runs by OVer for leading batsman

In [ ]:
# identify top key
key='batsman'
topbatsmen=df_bbb[key].value_counts()[:50].index

In [ ]:
df_=df_bbb.pivot_table(index=['batsman','over'],aggfunc=['count','sum'],values='batsman_runs'
                  )

df_=df_.fillna(0).reset_index()
df_.columns=['batsman','over','balls','runs']
df_['strike_rate']=round(df_.runs/df_.balls*100,1)
df_[df_.batsman.isin(topbatsmen)
   ].pivot_table(columns='over',index='batsman',values='strike_rate',aggfunc='mean').fillna(0
# ).head(200
        ).style.format("{:.1f}").background_gradient(cmap=cm).apply(highlight_max)


In [ ]:
alt.data_transformers.disable_max_rows()

df_.columns
alt.Chart(df_[df_.batsman.isin(topbatsmen)]).mark_point(shape="circle").encode(
alt.X('over'),
alt.Y('mean(strike_rate)'),
color='batsman',
text="batsman",
size='balls',
opacity='runs',
tooltip=list(df_.columns)
).properties(width=600,height=500)